In [4]:
!nvidia-smi

Sat Jul 22 17:46:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   31C    P0    52W / 300W |  16245MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   

In [5]:
import os
from rosemary import jpt_setup; jpt_setup()

from transformers.trainer_utils import get_last_checkpoint

from llm.submit import (
    multiline_to_singleline,
    submit_job_ccc,
    submit_job_aimos,
    submit_job,
    get_run_statistics)
import pandas as pd
import json
import platform
import tempfile
import subprocess
import shlex

arch = platform.uname().processor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem usage (GB)', 'per-epoch time (hr)', 'per-iter time (s)']

# 55k data points, batch_size=128
data_oasst1 = [
    ('gpt2', 0.124, 'bf16', 'no', 10, None, None),
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 2.5, 11),
    # incorporate deep speed is costly!
    ('gpt2-Large', 0.774, 'bf16', 'stage 3 no offloading', 40, 6, 25),
    # 1 a100_40g: without offloading OOM on `.backward()`, runs fine with offloading.
    ('gpt2-xl', 1.5, 'bf16', 'stage 3 with offloading', 40, 13, 55),
    # 4 v100_32g: without offloading.
]

df_oasst1 = pd.DataFrame(data_oasst1, columns=cols)

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem (GB)', 'cpu mem (GB)', 'per-epoch time (hr)', 'per-iter time (s)']
data = [
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 7, 9.5, 9),
]

print('instruction tune human-mix on 1 a100_40g:')
df = pd.DataFrame(data, columns=cols)
df

instruction tune human-mix on 1 a100_40g:


,model,size,mixed-precision,deepspeed,gpu mem (GB),cpu mem (GB),per-epoch time (hr),per-iter time (s)
0,gpt2-Large,0.774,bf16,no,36,7,9.5,9


# Finetuning with openinstruct/finetune.py


In [8]:
shell_scripts_template = """
conda activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

echo "Running on $(hostname)"
echo "======"
echo {cmd}
echo "======"

{cmd}

[ ! -f "{log_dir}/${{LSB_JOBID}}*.out" ] || mv "{log_dir}/${{LSB_JOBID}}*.out" "{save_dir}"
"""

In [ ]:
# job_name = 'ft'
# test_run = 1
# test_run = bool(test_run)

# queue = 'x86_12h' # 'x86_12h'
# num_cpus = 20
# num_gpus = 1
# cpu_mem = 32
# require = 'a100_80gb'

# # model_name_or_path = 'mosaicml/mpt-7b'; max_seq_length = 2048
# # model_name_or_path = 'gpt2'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-Large'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-xl'; max_seq_length = 1024
# model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048


# train_file = 'data/processed/oasst1/oasst1_data.jsonl'; train_file_short = 'oasst1'
# train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
# # train_file = 'data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl'; train_file_short = 'human_mix_shuffled'

# output_dir = f"results/{model_name_or_path.replace('/', ':')}_{train_file_short}"
# if test_run:
#     output_dir = 'jpt_' + output_dir

# use_deepspeed = False
# # deepspeed_config_file = 'ds_configs/stage3_no_offloading_accelerate_setauto.conf'
# # deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate.conf'
# deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate_setauto.conf'

# use_lora = True
# lora_rank = 4
# lora_alpha = lora_rank
# lora_dropout = 0.05

# batch_size_per_gpu = 1
# total_batch_size = 128
# mixed_precision = 'bf16' # 'bf16', 'fp16'
# checkpointing_steps = None # every n steps, where n='1' or every 'epoch'

# gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

# print(f"Training {model_name_or_path} "
#       f"using {num_gpus} GPUs, "
#       f"{batch_size_per_gpu} batch size per GPU, "
#       f"{gradient_acc_steps} gradient accumulation steps.")

# # do use fast tokenizer since mpt-7b does not have a fast tokenizer counter-part
# #     --use_slow_tokenizer \
# # do not use flash attention, since having problem installing flash-attn with cuda 12.1
# #     --use_flash_attn \

# cmd = f"""
# {'!cd .. && ' if test_run else ''}accelerate launch \
#     --mixed_precision {mixed_precision} \
#     --num_machines 1 \
#     --num_processes {num_gpus} \
#     {'--use_deepspeed' if use_deepspeed else ''}
#     {'--deepspeed_config_file '+deepspeed_config_file if use_deepspeed else ''}
#     open_instruct/finetune.py \
#     --model_name_or_path {model_name_or_path} \
#     --tokenizer_name {model_name_or_path} \
#     --train_file {train_file} \
#     --max_seq_length {max_seq_length} \
#     {'--use_lora' if use_lora else ''}
#     --lora_rank {lora_rank} \
#     --lora_alpha {lora_alpha} \
#     --lora_dropout {lora_dropout} \
#     --preprocessing_num_workers 16 \
#     --per_device_train_batch_size {batch_size_per_gpu} \
#     --gradient_accumulation_steps {gradient_acc_steps} \
#     --learning_rate 2e-5 \
#     --lr_scheduler_type linear \
#     --warmup_ratio 0.03 \
#     --weight_decay 0. \
#     --num_train_epochs 2 \
#     --output_dir {output_dir} \
#     --with_tracking \
#     --report_to tensorboard \
#     {'--checkpointing_steps '+str(checkpointing_steps) if checkpointing_steps else ''}
#     --logging_steps 1
# """

# # things to test to see its effects on (1) eval perf (2) runtime.
# #
# # - int8
# # - mixed_precision bf16 or no
# # - with/without LoRA
# # - LoRA's rank/alpha (alpha typically set to 2*rank)
# # - batch size
# # - micro-batch size (largest without running out of memory)


# cmd = multiline_to_singleline(cmd)
# if test_run:
#     print()
#     print(cmd)

# shell_scripts = shell_scripts_template.format(
#     cmd=cmd,
#     log_dir=os.getcwd(),
#     save_dir=output_dir)
# out = submit_job_ccc(
#     shell_scripts, 
#     job_name=job_name, 
#     queue=queue,
#     num_cpus=num_cpus,
#     cpu_mem=cpu_mem,
#     require=require,
#     num_gpus=num_gpus,
#     test_run=test_run,
# )
# if not test_run:
#     print(out)

In [ ]:
# llama7b+lora, micro-bsz=1, bsz=128, 
#     66gb gpu mem, in it 28gb for torch tensor. so fit on 1 a100_80gb

#   1%|▏         | 57/4222 [22:24<25:53:47, 22.38s/it]07/11/2023 23:13:55 - INFO - 
# __main__ -   Step: 57, LR: 9.047619047619049e-06, Loss: 2.2924644947052
# 


!cd .. && accelerate launch --mixed_precision bf16 --num_machines 1 --num_processes 1 open_instruct/finetune.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --use_lora --lora_rank 4 --lora_alpha 4 --lora_dropout 0.05 --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 128 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --num_train_epochs 2 --output_dir results/huggyllama:llama-7b_human_mixjpt_results/huggyllama:llama-7b_human_mix --with_tracking --report_to tensorboard --logging_steps 1





# Finetuning with openinstruct/finetune_trainer.py


In [ ]:
# runtime:
# shared: humanmix, max_sequence_length=2048.
# just a single process if num_gpus=1 otherwise use python fsdp with gradient-checkpointing.
#
# llama7b, micro-bsz=2, grad-ckpt, 2xa100_80gb: 27s/it, 32hrs
# 
# llama7b+lora(r=4),   micro-bsz=1, no-grad-ckpt,a100_80gb: 27s/it, 32hrs, 66gb gpu mem
# llama7b+lora(r=16),  micro-bsz=1, no-grad-ckpt,a100_80gb: 24s/it, 28hrs, 67gb gpu mem
# llama7b+lora(r=256), micro-bsz=1, no-grad-ckpt,a100_80gb: 24s/it, 28hrs, 70gb gpu mem
#
# llama7b+lora(r=4),   micro-bsz=1,  grad-ckpt,  a100_80gb: 32s/it, 38hrs
# llama7b+lora(r=4),   micro-bsz=2,  grad-ckpt,  a100_80gb: 32s/it, 38hrs (+20% runtime with grad-ckpt)
# llama7b+lora(r=4),   micro-bsz=16, grad-ckpt,  a100_80gb: 68s/it, 80hrs, 75gb gpu mem
# llama7b+lora(r=128), micro-bsz=16, grad-ckpt,  a100_80gb: 70s/it, 82hrs
# 
# take-aways
# - larger batch should speed up training. however, it also implies that the batch 
#   will be padded to longer sequence length, due to chat data, thereby increasing
#   compute required to process a batch. Therefore, micro-bsz=1 seems to be ok.
#


In [9]:
job_name = 'ft-trainer'
test_run = 1
test_run = bool(test_run)
# specify `job_duration` to chain jobs for >12hr jobs.
job_duration = 6 if arch == 'ppc64le' else 12
# shell_scripts_modification_fn = lambda x: x.replace('--overwrite_output_dir ', '')

num_cpus = 32
cpu_mem = 64
num_gpus = 1; gpu_type = 'a100_80gb'
num_gpus = 6; gpu_type = 'v100'

overwrite_output_dir = True if test_run else False # always continue from ckpt if run from cluster.
save_strategy = 'steps'
save_steps = 100

model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048
# model_name_or_path = 'gpt2'; max_seq_length = 1024
train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; abbr_train_file = 'humanmix'

num_train_epochs = 2
batch_size_per_gpu = 1
total_batch_size = 128
gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)


fsdp = False if num_gpus == 1 else "full_shard auto_wrap"
if 'gpt2' in model_name_or_path: fsdp_transformer_layer_cls_to_wrap = 'GPT2Block'
elif 'llama' in model_name_or_path: fsdp_transformer_layer_cls_to_wrap = 'LlamaDecoderLayer'
else: raise ValueError('Not sure how to set `fsdp_transformer_layer_cls_to_wrap`')

use_lora = False
lora_rank = 8 # test {8, 16, 32, 128} # just [128, 8] for now.
lora_alpha = lora_rank
lora_dropout = 0.05
gradient_checkpointing = True

load_in_8bit = False

abbr_model_name = model_name_or_path.replace('/', ':')
if use_lora:
    abbr_model_name += f'+lora(r={lora_rank},a={lora_alpha})'
output_dirname = f"{abbr_model_name}_{abbr_train_file}"
if test_run:
    output_dirname = 'jpt_'+output_dirname
output_dir = os.path.join('results', output_dirname)


print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{batch_size_per_gpu} batch size per GPU, "
      f"{gradient_acc_steps} gradient accumulation steps.")

exe = 'python' if num_gpus == 1 else f"torchrun --nproc_per_node={num_gpus} --master_port=10002"

cmd = f"""
{'!cd .. && ' if test_run else ''}{exe}
    open_instruct/finetune_trainer.py \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    {'--load_in_8bit' if load_in_8bit else ''} \
    --use_fast_tokenizer True \
    --train_file {train_file} \
    --max_seq_length {max_seq_length} \
    {'--use_lora' if use_lora else ''}
    {'--lora_rank '+str(lora_rank) if use_lora else ''}
    {'--lora_alpha '+str(lora_alpha) if use_lora else ''}
    {'--lora_dropout '+str(lora_dropout) if use_lora else ''}
    --do_train \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size {batch_size_per_gpu} \
    --gradient_accumulation_steps {gradient_acc_steps} \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    --evaluation_strategy "no" \
    --logging_steps 1 \
    --save_strategy {save_strategy} \
    --save_steps {save_steps} \
    --save_total_limit 1 \
    --num_train_epochs {num_train_epochs} \
    {'--fsdp "'+fsdp+'"' if fsdp else ''}
    {'--fsdp_transformer_layer_cls_to_wrap "'+fsdp_transformer_layer_cls_to_wrap+'"' 
        if fsdp else ''}
    {'--gradient_checkpointing' if gradient_checkpointing  else ''}
    {'--bf16 True' if arch == 'x86_64' else '--fp16 True'} \
    {'--tf32 True' if arch == 'x86_64' else ''} \
    --report_to tensorboard \
    --torch_dtype {'bfloat16' if arch=='x86_64' else 'float16'} \
    --dataloader_num_workers 8 \
    {'--overwrite_output_dir' if overwrite_output_dir else ''} \
    --output_dir "{output_dir}"
"""


cmd = multiline_to_singleline(cmd)
if test_run:
    print()
    print(cmd)

shell_scripts = shell_scripts_template.format(
    cmd=cmd,
    log_dir=os.getcwd(),
    save_dir=output_dir)
out = submit_job(
    shell_scripts, 
    job_name=job_name, 
    num_cpus=num_cpus,
    cpu_mem=cpu_mem,
    num_gpus=num_gpus,
    gpu_type=gpu_type,
    test_run=test_run,
    job_duration=job_duration,
)
if not test_run:
    print(out)

Training huggyllama/llama-7b using 6 GPUs, 1 batch size per GPU, 21 gradient accumulation steps.

!cd .. && torchrun --nproc_per_node=6 --master_port=10002 open_instruct/finetune_trainer.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --use_fast_tokenizer True --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --do_train --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 21 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --evaluation_strategy "no" --logging_steps 1 --save_strategy steps --save_steps 100 --save_total_limit 1 --num_train_epochs 2 --fsdp "full_shard auto_wrap" --fsdp_transformer_layer_cls_to_wrap "LlamaDecoderLayer" --gradient_checkpointing --fp16 True --report_to tensorboard --torch_dtype float16 --dataloader_num_workers 8 --overwrite_output_dir --output_dir "results/jpt_huggyllama:llama-7b_humanmix"

Submiting job with:
{
    "job

<function torch.distributed.elastic.multiprocessing.errors.record(fn: Callable[..., ~T], error_handler: Optional[torch.distributed.elastic.multiprocessing.errors.error_handler.ErrorHandler] = None) -> Callable[..., ~T]>

In [19]:

!cd .. && 
LOGLEVEL=INFO TORCHELASTIC_ERROR_FILE='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/errfile' torchrun --nproc_per_node=6 --master_port=10002 open_instruct/finetune_trainer.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --use_fast_tokenizer True --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --do_train --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 21 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --evaluation_strategy "no" --logging_steps 1 --save_strategy steps --save_steps 100 --save_total_limit 1 --num_train_epochs 2 --fsdp "full_shard auto_wrap" --fsdp_transformer_layer_cls_to_wrap "LlamaDecoderLayer" --gradient_checkpointing --fp16 True --report_to tensorboard --torch_dtype float16 --dataloader_num_workers 8 --overwrite_output_dir --output_dir "results/jpt_huggyllama:llama-7b_humanmix"


*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : open_instruct/finetune_trainer.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 6
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:10002
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 0
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_iph19hz6/none_8c547g5h
INFO:torch.distributed.elastic.agent.server.api:[default] starting workers for entrypoint: python3.10
INFO:torch.distributed.elastic.ag

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 143.77it/s]
07/22/2023 18:14:42 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-6c4472be975c5495/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 124.05it/s]
07/22/2023 18:14:42 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-6c4472be975c5495/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
07/22/2023 18:14:42 - INFO - datasets.info - Loading Dataset info from /gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-6c4472be975c5495/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 124.51it/s]
[INFO|configuration_utils.py:712] 2023-07-22 18:14:42,

# conda environments:
#
base                     /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3
open-instruct         *  /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct
wpq-llm                  /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/wpq-llm



# eval



In [ ]:
shell_scripts_template = """
source ~/.profile
conda activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

echo "Running on $(hostname)"
echo "======"
echo {cmd}
echo "======"

{cmd}

[ ! -f "{log_dir}/${{LSB_JOBID}}.out" ] || mv "{log_dir}/${{LSB_JOBID}}.out" "{save_dir}"
"""

In [ ]:
task_name = 'mmlu'
task_name = 'gsm'
task_name = 'bbh_s=0'
task_name = 'bbh_s=3'
# task_name = 'humaneval'
job_name = f'eval.{task_name}'

test_run = 1
test_run = bool(test_run)
use_chat_format = True
eval_final_model = True

batch_size = 10
if task_name == 'gsm':
    queue = 'x86_1h' # 10min for n=200
if task_name == 'bbh_s=0':
    queue = 'x86_1h'
if task_name == 'bbh_s=3':
    queue = 'x86_12h'
    batch_size = 5 # for longer prompts.
if task_name == 'mmlu':
    queue = 'x86_1h'
    batch_size = 10
if task_name == 'humaneval':
    queue = 'x86_1h' # pass@1: 10min, pass@10: 100min
    batch_size = 10
    
num_cpus = 10
cpu_mem = 32 # mem usage quite small for llama7b+lora on bbh

load_in_8bit = False

models = []
# models += ['t5-small', 't5-base', 't5-large', 't5-3b', 't5-11b']
# models += ['t5-11b']
# models += ['google/flan-t5-small', 'google/flan-t5-base', 'google/flan-t5-large', 'google/flan-t5-xl', 'google/flan-t5-xxl']
# models += ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
models += [os.path.join('results/baselines', x) for x in [
    'huggyllama/llama-7b',  # , 'mosaicml/mpt-7b'
]]

# models += ['results/huggyllama:llama-7b_human_mix-trainer_savebystep/']
# models = [glob.glob(os.path.join(x, 'checkpoint*')) for x in models]
# models = [x for l in models for x in l]

models += [x if eval_final_model else get_last_checkpoint(x) for x in [
#     'results/huggyllama:llama-7b+lora(r=128,a=128)_humanmix',
    'results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix',
#     'results/huggyllama:llama-7b_humanmix',
]]


info = {}  
cmds = []
for model_name_or_path in models:
    run_id = model_name_or_path
    save_dir = f'{model_name_or_path}/eval/{task_name}'
    if use_chat_format:
        save_dir += '_chatfmt'
    
    if task_name == 'mmlu':
        cmd = f"""
        python -m eval.mmlu.run_eval \
            --data_dir data/eval/mmlu \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --ntrain 0 \
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name == 'gsm':
        cmd = f"""
        python -m eval.gsm.run_eval \
            --data_dir data/eval/gsm/ \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --max_num_examples 200 \
            --n_shot 8 \
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name.startswith('bbh'):
        cmd = f"""
        python -m eval.bbh.run_eval \
            --data_dir data/eval/bbh/ \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            {'--no_cot' if 's=0' in task_name else ''}
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name.startswith('humaneval'):
        cmd = f"""
        python -m eval.codex_humaneval.run_eval \
            --data_file data/eval/codex_humaneval/HumanEval.jsonl.gz \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --eval_pass_at_ks 1 \
            --unbiased_sampling_size_n 1 \
            --temperature 0.1 \
            {'--use_chat_format' if use_chat_format else ''}
            {'--load_in_8bit' if load_in_8bit else ''}
        """
        
    cmd = multiline_to_singleline(cmd)
    cmds.append(cmd)
    print(cmd)
    
    # submit
    shell_scripts = shell_scripts_template.format(
        cmd=cmd,
        log_dir=os.getcwd(),
        save_dir=save_dir)
    out = submit_job_ccc(
        shell_scripts, 
        job_name=job_name, 
        queue=queue,
        num_cpus=num_cpus,
        cpu_mem=cpu_mem,
        num_gpus=1,
        test_run=test_run,
    )
#     if test_run: print(out['jbsub_cmd'])
    
    

In [ ]:
class EvalResults:
    
    def __init__(self, save_dir, run_name=None):
        self.save_dir = save_dir
        self.eval_dir = os.path.join(self.save_dir, 'eval')
        self.run_name = run_name if run_name else self.save_dir
        
    def get_result_df(self):

        task_names = os.listdir(self.eval_dir)
        print(task_names)

        dfs = []
        for task_name in task_names:
            task_save_dir = os.path.join(self.eval_dir, task_name)
            metrics_file = os.path.join(task_save_dir, 'metrics.json')
            if not os.path.exists(metrics_file):
                continue
            with open(metrics_file, 'r') as f:
                metrics = json.load(f)

            if 'mmlu' in task_name:
                for k, v in metrics['subcat_acc'].items():
                    metrics[k] = v
                del metrics['subcat_acc']
                for k, v in metrics['cat_acc'].items():
                    metrics[k] = v
                del metrics['cat_acc']

            columns = [f'{task_name}/{k}' for k in metrics.keys()]
            # columns = pd.MultiIndex.from_tuples(columns)
            data = list(metrics.values())

            df = pd.DataFrame([data], columns=columns)
            dfs.append(df)

        df = pd.concat(dfs, axis=1)
        mapper = {
            'mmlu/average_acc': 'MMLU/0-shot',
            'mmlu_chatfmt/average_acc': 'MMLU/0-shot_chatfmt',
            'gsm/exact_match': 'GSM/CoT',
            'gsm_chatfmt/exact_match': 'GSM/CoT_chatfmt',
            'bbh_s=0/average_exact_match': 'BBH/Direct',
            'bbh_s=0_chatfmt/average_exact_match': 'BBH/Direct_chatfmt',
            'bbh_s=3/average_exact_match': 'BBH/CoT', 
            'bbh_s=3_chatfmt/average_exact_match': 'BBH/CoT_chatfmt',
            'humaneval/pass@1': 'Codex-Eval/Pass@1',
            'humaneval_chatfmt/pass@1': 'Codex-Eval/Pass@1_chatfmt',
        }
        cols = [col for col in mapper.keys() if col in df]
        for col in cols:
            df[col] = df[col]*100
        df = df[cols]
        df = df.rename(columns=mapper)
        df.insert(0, 'Model', [self.run_name])
        return df

# get_last_checkpoint(v)
save_dirs = []
save_dirs += [
    ('llama-7b', '../results/baselines/huggyllama/llama-7b/'),
    ('llama-7b+humanmix', '../results/huggyllama:llama-7b_humanmix'),
    ('llama-7b+lora(r=8,a=8)', '../results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix'),
    ('llama-7b+lora(r=128,a=128)', '../results/huggyllama:llama-7b+lora(r=128,a=128)_humanmix'),
]


dfs = []
for model_name, save_dir in save_dirs:
    r = EvalResults(save_dir, model_name)
    df = r.get_result_df()
    dfs.append(df)

pd.set_option('display.max_colwidth', None)

df = pd.concat(dfs, axis=0)
with pd.option_context('display.float_format', '{:0.1f}'.format):
    dfc = df.copy()
    cols = [x.split('_') for x in df.columns]
    cols = [x+[''] if len(x)==1 else x for x in cols]
    dfc.columns = pd.MultiIndex.from_tuples(cols)
    display(dfc)

In [ ]:
with pd.option_context('display.float_format', '{:0.1f}'.format):
    display(df[['Model']+[x for x in df.columns if 'chatfmt' in x]])

In [ ]:
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(df[[x for x in df.columns if 'chatfmt' not in x]])


In [ ]:
import glob

models = []
models += ['t5-small', 't5-base', 't5-large', 't5-3b', 't5-11b']
models += ['huggyllama/llama-7b']
save_dirs = [f'../results/baselines/{x}/eval/gsm/' for x in models]

data = []
for model, save_dir in zip(models, save_dirs):
    logfile_path = glob.glob(os.path.join(save_dir, '*.out'))[0]
    out = get_run_statistics(logfile_path)
    with open(os.path.join(save_dir, 'metrics.json'), 'r') as f:
        metrics = json.load(f)
    data.append((model, out['cpu_time']/60/60, out['avg_mem'], out['max_mem'], metrics['exact_match']))
    

import pandas as pd
columns = ['name', 'cpu_time (hr)', 'avg_mem', 'max_mem', 'exact_match']
df = pd.DataFrame(data, columns=columns)
df

In [ ]:


df

